# Extract Respiration Signal with Big-Small Model

This notebook demonstrates how to extract the respiration signal from a video using the Big-Small model.

In [ ]:
import respiration.dataset as repository

dataset = repository.from_default()

subject = 'Proband05'
scenario = '101_natural_lighting'

In [ ]:
frames, meta = dataset.get_video_rgb(
    subject, 
    scenario,
    num_frames=300, 
    show_progress=True)

In [ ]:
import respiration.utils as utils

device = utils.get_torch_device()
device

In [ ]:
from respiration.extractor.big_small import *

model, key_matching = load_model(device=device)
key_matching

In [ ]:
model.eval()

In [ ]:
from respiration.extractor.big_small import *

# Get the first 10 seconds of the video
big, small = preprocess_frames(frames)

In [ ]:
# Show the first big and small frame
import matplotlib.pyplot as plt

_, axs = plt.subplots(1, 2, figsize=(15, 5))

axs[0].imshow(small[1])
axs[0].set_title('Small Frame')

axs[1].imshow(big[1])
axs[1].set_title('Big Frame')

In [ ]:
big.shape, small.shape

In [ ]:
# Convert the frames to a tensor
small_tensor = torch.tensor(small, device=device)
big_tensor = torch.tensor(big, device=device)

In [ ]:
small_tensor.shape, big_tensor.shape

In [ ]:
# Transform the tensor to the shape expected by the model (frame_count, c, w, h)
small_tensor = small_tensor.permute(0, 3, 1, 2)
big_tensor = big_tensor.permute(0, 3, 1, 2)

small_tensor.shape, big_tensor.shape

In [ ]:
# Extract the signals
with torch.no_grad():
    au_out, bvp_out, resp_out = model((big_tensor, small_tensor))

resp_out.shape

In [ ]:
import respiration.preprocessing as preprocessing

waveform = resp_out.cpu().numpy().squeeze()
waveform = preprocessing.standard_processing(waveform, meta.fps)

In [ ]:
# Plot the rPPG signal
plt.figure(figsize=(20, 5))
plt.plot(waveform)
plt.title('Respiration Signal')
plt.xlabel('Frame')
plt.ylabel('Amplitude')
plt.show()